In [37]:
# read dataset one image
from keras.preprocessing import image

imageSize = 128

def readImage(path):
    img = image.load_img(path, target_size=(imageSize, imageSize))
    x = image.img_to_array(img)
    return x

In [38]:
def getLabel(path):
    name = path[path.rfind('/') + 1:path.index('.')]
#     print (name)
    return 1 if name == 'dog' else 0

In [56]:
import helper

def readFeatureAndLabel():
    train_files = helper.get_train_files()
    file_count = len(train_files)
    train_feature = []
    train_label = []
    for i in range(file_count):
        f = train_files[i]
#         print (f)
        train_feature.append(readImage(f))
        train_label.append(getLabel(f))
    
    return train_feature, train_label

In [55]:
import numpy as np

total_feature, total_label = readFeatureAndLabel()
total_feature = np.array(total_feature)
total_feature = total_feature.astype('float32')
total_feature /= 255

total_label = np.array(total_label)
# print (total_label)

25000


AttributeError: 'list' object has no attribute 'size'

In [41]:
# split train_feature, train_label, validation_feature, validation_label
from sklearn.model_selection import train_test_split
train_feature, validation_feature, train_label, validation_label = train_test_split(total_feature, total_label, test_size=0.1, random_state=1)
    
# print (train_label)

In [46]:
import math

def score_fun(label_pre):
    total_score = 0
    size = validation_label.size
    for i in range(size):
        y = validation_label[i]
        yp = label_pre[i]
#         print (str(y) + " " + str(yp))
        try:
            total_score = y * math.log(yp) + (1 - y)*math.log(1-yp)
        except:
            print (str(i) + ": " + str(y) + " - " + str(yp))
    return -total_score/size

In [48]:
# convolutional
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(imageSize, imageSize, 3)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(train_feature, train_label, batch_size=16, epochs=10, validation_data=(validation_feature, validation_label))

pre_val_label = model.predict(x=validation_feature, batch_size=16)
for i in range(pre_val_label.size):
    print(str(i) + ": " + str(pre_val_label[i]))
score = model.evaluate(validation_feature, validation_label, batch_size=16)
print (score)


Train on 921 samples, validate on 103 samples
Epoch 1/10
921/921 [==============================] - 87s 94ms/step - loss: 5.9990 - acc: 0.4875 - val_loss: 0.6935 - val_acc: 0.4660
Epoch 2/10
921/921 [==============================] - 88s 96ms/step - loss: 0.7108 - acc: 0.5266 - val_loss: 0.7076 - val_acc: 0.4369
Epoch 3/10
921/921 [==============================] - 84s 91ms/step - loss: 0.6971 - acc: 0.5635 - val_loss: 1.2470 - val_acc: 0.5340
Epoch 4/10
921/921 [==============================] - 81s 88ms/step - loss: 0.7055 - acc: 0.5874 - val_loss: 0.7404 - val_acc: 0.4951
Epoch 5/10
921/921 [==============================] - 84s 91ms/step - loss: 0.6507 - acc: 0.6232 - val_loss: 0.7375 - val_acc: 0.5340
Epoch 6/10
921/921 [==============================] - 81s 88ms/step - loss: 0.6150 - acc: 0.6743 - val_loss: 0.8514 - val_acc: 0.5631
Epoch 7/10
921/921 [==============================] - 85s 92ms/step - loss: 0.5827 - acc: 0.7047 - val_loss: 0.8145 - val_acc: 0.5728
Epoch 8/10
921/9

In [49]:
score = score_fun(pre_val_label)

print (score)

0.011442315472113699
